In [1]:
import gym
import time
from Entity import Entity
from EvolutionCtrl import Population_Manager

# 1) Environment
env = gym.make("BipedalWalker-v3") 
env.seed(0)    # set seed for const. initial state

# 2) Training Params
POP_SIZE = 120
MAX_SEQUENCE_LEN = 400
N_GENERATIONS = 1500    

weight_interval = [-1, 1]                  # interval for initial and random weights
print(env.action_space.shape)
n_layer_nodes = [env.observation_space.shape[0], 48, 48, 32, env.action_space.shape[0]]
print(env.observation_space.shape)
init_population = [Entity(n_layer_nodes, weight_interval) for i in range(POP_SIZE)]  
pop_manager = Population_Manager(init_population, .1, .1)

(4,)
(24,)
C:\Users\Mars\Anaconda3\envs\gym\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


TypeError: __init__() takes 4 positional arguments but 6 were given

In [6]:
def agent_env_loop(entity):
        entity.survived = True
        entity.fitness = 0
        env.seed(10) 
        observation = env.reset()                                            # s_0
        for i in range(MAX_SEQUENCE_LEN):  
            action = entity.controller.feed_forward(observation)             # a_t, s_t            
            observation, reward, done, _ = env.step(action)                  # s_{t+1}, r_{t+1}, a_t       
            entity.fitness += reward
            if done:
                break   

# Optimize 
for i in range(N_GENERATIONS):
    t_start = time.time() 
    for entity in pop_manager.population:
        if not entity.survived:
            agent_env_loop(entity)
    
    pop_manager.breed_new_population()
    print(f"Generation {i}: {time.time() - t_start}")


.086296081542969
66 survived walker, max fitness is: 51.38218996354512
Generation 1277: 4.767001628875732
64 survived walker, max fitness is: 51.38218996354512
Generation 1278: 5.477001905441284
61 survived walker, max fitness is: 51.38218996354512
Generation 1279: 5.976996898651123
63 survived walker, max fitness is: 51.38218996354512
Generation 1280: 5.677081346511841
58 survived walker, max fitness is: 51.38218996354512
Generation 1281: 5.583029508590698
60 survived walker, max fitness is: 51.38218996354512
Generation 1282: 6.3389668464660645
60 survived walker, max fitness is: 51.38218996354512
Generation 1283: 6.159034729003906
64 survived walker, max fitness is: 51.38218996354512
Generation 1284: 5.749997854232788
57 survived walker, max fitness is: 51.38218996354512
Generation 1285: 5.645968437194824
61 survived walker, max fitness is: 51.38218996354512
Generation 1286: 6.665130376815796
57 survived walker, max fitness is: 51.38218996354512
Generation 1287: 5.182026624679565
68 

In [4]:
pop_manager.population.sort(key=lambda x: x.fitness, reverse=True) 
best_agent = pop_manager.population[-50]

env.seed(10) 
observation = env.reset()
for _ in range(MAX_SEQUENCE_LEN):
    env.render()
    action = best_agent.controller.feed_forward(observation)            # a_t, s_t            
    observation, reward, done, _ = env.step(action)                     # s_{t+1}, r_{t+1}, a_t       
env.close()

In [35]:
class testclass(object):
    def __init__(self):
        self.weights = [[1,2,3], [4,5,6]]

x = testclass()

In [36]:
y = x.weights[0]
y

[1, 2, 3]

In [37]:
y[0] = 7

In [38]:
x.weights[0]

[7, 2, 3]

In [39]:
y

[7, 2, 3]

In [34]:
a = [1,2,3]
b = a
b[0] = 4
a

[4, 2, 3]